In [10]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import os

data_path = 'E:/ml-100k/u.data'  # Your file path
try:
    ratings = pd.read_csv(data_path, sep='\t', names=['user_id', 'item_id', 'rating', 'timestamp'], encoding='utf-8')
except FileNotFoundError:
    print(f"Error: File {data_path} not found. Ensure the MovieLens 100k dataset is downloaded and extracted to the correct path.")
    raise
    
utility_matrix = ratings.pivot(index='user_id', columns='item_id', values='rating')
user_means = utility_matrix.mean(axis=1)
centralized_matrix = utility_matrix.sub(user_means, axis=0).fillna(0)

centralized_matrix_T = centralized_matrix.T
item_95_profile = centralized_matrix[95].values.reshape(1, -1)
item_similarities = cosine_similarity(centralized_matrix_T)
item_95_sim = item_similarities[94]
similar_items = np.argsort(item_95_sim)[::-1][1:11]

user_200_ratings = utility_matrix.loc[200].fillna(0)
user_15_ratings = utility_matrix.loc[15].fillna(0)

def predict_rating(user_ratings, similar_items, item_similarities):
    weights = item_similarities[similar_items]
    ratings = user_ratings.iloc[similar_items + 1]
    valid = ratings != 0
    if valid.sum() == 0:
        return 0
    weighted_sum = np.sum(ratings[valid] * weights[valid])
    weight_sum = np.sum(weights[valid])
    return weighted_sum / weight_sum if weight_sum != 0 else 0

pred_200_95 = predict_rating(user_200_ratings, similar_items, item_95_sim)
pred_15_95 = predict_rating(user_15_ratings, similar_items, item_95_sim)
user_200_profile_user_space = centralized_matrix.loc[200].values.reshape(1, -1)
user_15_profile_user_space = centralized_matrix.loc[15].values.reshape(1, -1)
print(f"Predicted rating for user 200 on item 95: {pred_200_95:.4f}")
print(f"Predicted rating for user 15 on item 95: {pred_15_95:.4f}")

recommended_user = 200 if pred_200_95 > pred_15_95 else 15
print(f"The recommender system will recommend movie 95 to user {recommended_user} due to higher predicted rating.")

Predicted rating for user 200 on item 95: 4.1569
Predicted rating for user 15 on item 95: 0.0000
The recommender system will recommend movie 95 to user 200 due to higher predicted rating.
